In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# hyperparameters
lr = 0.001
training_iters = 100000
batch_size = 128

n_inputs = 28 # 每一次input一行pixel,即n_inputs = 28
n_steps = 28 # 一共28列,因此n_steps = 28
n_hidden_units = 128 # neurons in hidden layer
n_classes = 10

In [4]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

In [8]:
# Define weights and biases
weights = {
    # (28, 128)
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    # (128, 10)
    'out': tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units,])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes,]))
}

In [12]:
def RNN(X, weights, biases):
    # hidden layer for inut to cell
    # X(128 batch, 28 steps, 28 inputs) ==> X(128 * 28, 28 inputs)
    # 将展成一个长度为28的向量,一共128 * 28个(reshape中第一个参数为-1,表示个数由总数除以shape得到)
    X = tf.reshape(X, [-1, n_inputs])
    # X_in : (128 batch * 28 steps, 128 hidden)
    X_in = tf.matmul(X, weights['in']) + biases['in']
    # X_in ==> (128 batch, 28 steps, 128 hidden)
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])
    
    # LSTM cell
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units)
    _init_state = cell.zero_state(batch_size, dtype=tf.float32)
    # dynamic RNN比传统RNN具有更好的效果
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=_init_state, time_major=False)
    
    # hidden layer for output as the final results
    results = tf.amtmul(states[1], weights['out']) + biases['out']
    return results